### What resource we need to Train this large data??: GPU

### Completed Tasks:

--> GEE Asset/Crop polygons

--> polygon area estimation for each crop type data

--> Plante/NICFI Pixel count for each crops, each polygons

--> Graphic representation for area vs Pixel count

### Next tasks:

--> Cloud masking for Planet (Algorthms)

--> Image Normalization: Scale the pixel values (e.g., 0-255) to a range that is typical for neural network inputs, such as 0-1 or -1 to 1.

--> Data Augmentation: To increase the diversity of the training data and prevent overfitting, apply transformations like rotations, translations, scaling, and horizontal flipping.

--> Patch Extraction: For high-resolution images, it might be necessary to create smaller, manageable patches. This makes the training process more efficient and helps in handling large images during deployment.

--> DL (FCNN) Trainings
--> Parallel ML training

### Reading corner about crop mapping in Senegal
SEN4STAT/ESA: https://www.esa-sen4stat.org/user-stories/senegal-prototype/

EOSTAT/FAO: https://data.apps.fao.org/catalog/dataset/5c377b2b-3c2e-4b70-afd7-0c80900b68bb/resource/50bc9ff5-95d2-40cd-af12-6aee2cfcc4ae

In [ ]:
#Lib imports:
import ee
#print('Using EE version ', ee.__version__)
import folium
#print('Using Folium version ', folium.__version__)
from os import MFD_HUGE_1MB
import pandas as pd
import time
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
#import cv2
import tensorflow as tf
from tensorflow import keras
from keras import layers
#print('Using TF version ', tf.__version__)
from typing import Dict, Iterable, List, Tuple
#from google.colab import auth


In [ ]:
# authenticate with user credentials
#auth.authenticate_user()

In [ ]:
# Authenticate to the Earth Engine servers
ee.Authenticate()

# Initialize the Earth Engine object with Google Cloud project ID
project_id = 'ee-kkidia3'
ee.Initialize(project=project_id)


In [ ]:
# @title Default title text

def list_ee_assets(folder_path):
    """ List all assets in the specified Earth Engine folder path """
    try:
        assets_list = ee.data.getList({'id': folder_path})
        for asset in assets_list:
            print(asset['id'])
    except Exception as e:
        print("Error accessing Earth Engine assets:", e)

# GEE Asset path
#folder_path = 'users/kkidia3'

# Call the function to list assets
#list_ee_assets(folder_path)


### Crop fields polygon for rainfed season 2018-2020

In [ ]:
#merge for the rainfed croping season 2018-2020.
merge18_20 = data_2018.merge(data_2019).merge(data_2020)



In [ ]:
# Import necessary librari
from google.colab import drive

# Function to calculate NDVI
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

# Load Sentinel-2 imagery and filter by date and bounds of the polygons
sentinel2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
    .filterBounds(data_2020).filterDate('2020-09-01', '2020-09-28') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))\
    .map(calculate_ndvi)

# Select bands to use for classification
bands = ['B2', 'B3', 'B4', 'B8', 'NDVI']

# Reduce the image collection to median to get a single composite image
image = sentinel2.median().select(bands)

# Function to sample data from the image and integrate with feature collection
def add_bands_to_fc(fc, image):
    def add_bands(feature):
        sampled = image.reduceRegion(           #Reduce the image collection to get a median composite image.
            reducer=ee.Reducer.mean(),
            geometry=feature.geometry(),
            scale=10,
            maxPixels=1e13
        )
        return feature.set(sampled)   #Create a function to add bands to the feature collection by sampling the image.
    return fc.map(add_bands)

# Integrate the bands into the feature collection
data_2020_with_bands = add_bands_to_fc(data_2020, image)  # using teh cleaned data -- can change to the original data

#Convert FeatureCollection to pandas DataFrame
def fc_to_df(fc):
    features = fc.getInfo()['features']
    dict_list = [f['properties'] for f in features]
    df = pd.DataFrame(dict_list)
    return df

# Display the first 5 rows of the dataframe
df = fc_to_df(data_2020_with_bands)
print(df.head())

# Export the new shapefile dataset to Google Drive
export_task = ee.batch.Export.table.toDrive(
    collection=data_2020_with_bands,
    description='data_2020_with_bands_sep',
    fileFormat='SHP'
)

export_task.start()

# Monitor the task status
while export_task.active():
    print('Polling for task (id: {}).'.format(export_task.id))
    time.sleep(30)

print('Export task completed with status:', export_task.status())

In [ ]:
# Function to get feature properties
def get_properties(feature):
    return ee.Feature(None, feature.toDictionary())

# Map the function over the collection
properties_list = data_2018.map(get_properties).getInfo()

# Extract properties from the list of features
properties_list = [feature['properties'] for feature in properties_list['features']]

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(properties_list)

# Rename columns if needed
#df.rename(columns={'Annee': 'Year'}, inplace=True) #df.rename(columns={'old_column_name': 'new_column_name'}, inplace=True)


# Display the DataFrame
df.head()

In [ ]:
# @title Crop_Ncrop

from matplotlib import pyplot as plt
import seaborn as sns
df.groupby('Crop_Ncrop').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
# @title Average Area by Crop Type

df.groupby('Speculatio')['Sup_ha'].mean().plot(kind='bar')

In [ ]:
# @title Admin1

from matplotlib import pyplot as plt
import seaborn as sns
df.groupby('Admin1').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
# Check for NaN values in each column
nan_counts = df.isna().sum()

# Display the columns with their respective NaN counts
print(nan_counts)

In [ ]:
# Display only columns with NaN values
nan_columns = nan_counts[nan_counts > 0]
print(nan_columns)


In [ ]:
# @title Default title text
# Define a function to add Earth Engine vector data as a layer to a Folium map
def add_ee_vector_layer(feature_collection, style, layer_name, map_object):
    painted = ee.Image().paint(feature_collection, 'constant', 2)  # Here 'constant' is a dummy property for visualization
    map_id_dict = painted.getMapId(style)
    folium.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; Google Earth Engine',
        name=layer_name,
        overlay=True,
        control=True
    ).add_to(map_object)

# Create a Folium map object
center = [14.4974, -14.4524]  # Center of the map (e.g., Senegal)
m1 = folium.Map(location=center, zoom_start=7)

# Styling for the vector layer
style = {
    'color': 'blue',  # Line color
    'fillColor': '00000000',  # Fill color with opacity (00)
}

# Add the merged crop fields to the map
add_ee_vector_layer(data_2018, style, 'Merged Crops 2019-2020', m1)

# Add a layer control panel to the map
folium.LayerControl().add_to(m1)

# Display the map
m1


### Polygons for FY2023

In [ ]:
# Function to load individual crop feature collections
def load_crop_data(asset_id):
    return ee.FeatureCollection(asset_id)


# Merge all crop fields into a single feature collection except fallow as not part of crop field
merged2023 = gnut.merge(gsorrel).merge(cashew).merge(cassava).merge(cowpea)\
                .merge(eggplant).merge(fonio).merge(gsorrel)\
                .merge(maize).merge(millet).merge(okra).merge(potato)\
                .merge(rice).merge(sesame).merge(sorgh).merge(soye).merge(squash).merge(taro)\
                .merge(vouandzou).merge(melon).merge(wheat).merge(milletmix).merge(gnutmix).merge(cowpmix)#.merge(fallow)

fallow = fallow.filter(ee.Filter.eq('class', 'Fallow')) #.merge(fallow) is not a crop



In [ ]:
print(merged2023.size().getInfo())

In [ ]:
print(fallow.size().getInfo())

Next steps:
- create shape file with bands and NDVI
- edit the clumen class of crop, fallow, N-crop etc
- export the classes and bands in to Google earht engine asset
- and test the new asset if band values and added class are in GEE ASSET

### Normalized bands : integrate the bands in to shapfle (Vector) 📂

To normalize the spectral bands, we'll create a function that normalizes each band by subtracting the minimum value and dividing by the range (max - min) of that band. We'll apply this function to each band in the image collection before calculating the NDVI and proceeding with the rest of the steps.

In [ ]:
# Function to normalize a band
def normalize_band(image, band_name, geometry):
    band = image.select(band_name)
    min_val = ee.Number(band.reduceRegion(ee.Reducer.min(), geometry, scale=10).get(band_name))
    max_val = ee.Number(band.reduceRegion(ee.Reducer.max(), geometry, scale=10).get(band_name))
    normalized_band = band.subtract(min_val).divide(max_val.subtract(min_val)).float()
    return image.addBands(normalized_band.rename(band_name + '_norm'), overwrite=True)

# Function to normalize all specified bands
def normalize_bands(image, band_names, geometry):
    for band_name in band_names:
        image = normalize_band(image, band_name, geometry)
    return image

# Function to calculate NDVI
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI') #(['B8_norm', 'B4_norm']).rename('NDVI')?????
    return image.addBands(ndvi)

# Load Sentinel-2 imagery and filter by date and bounds of the polygons
sentinel2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
    .filterBounds(gnut) \
    .filterDate('2023-09-01', '2023-09-30') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .map(lambda image: normalize_bands(image, ['B2', 'B3', 'B4', 'B8'], gnut.geometry())) \
    .map(calculate_ndvi)

# Select bands to use for classification
bands = ['B2_norm', 'B3_norm', 'B4_norm', 'B8_norm', 'NDVI', 'B2', 'B3','B4', 'B8'] #'NDVI_norm

# Reduce the image collection to median to get a single composite image
image = sentinel2.median().select(bands)

# Function to sample data from the image and integrate with feature collection
def add_bands_to_fc(fc, image):
    def add_bands(feature):
        sampled = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=feature.geometry(),
            scale=10,
            maxPixels=1e13,
            bestEffort=True
        )
        # Replace any null values with 0
        sampled = ee.Dictionary(sampled).map(lambda k, v: ee.Algorithms.If(v, v, ee.Number(0))) #????
        return feature.set(sampled)
    return fc.map(add_bands)

# Integrate the bands into the feature collection
crop_with_bands = add_bands_to_fc(gnut, image)

# Add new properties to each feature # edit the column or classes

def add_properties(feature):
    return feature.set('class', 'crop').set('sub_class', 'legume').set('year', '2023').set('region', '???')

merged2023_class_with_properties = crop_with_bands.map(add_properties)

# Get the first few features to display their properties
features_to_display = merged2023_class_with_properties.limit(5).getInfo()['features']

# Extract properties into a list of dictionaries
properties_list = [feature['properties'] for feature in features_to_display]

# Create a DataFrame


# Create a DataFrame from the properties list
df = pd.DataFrame(properties_list)

# Display the DataFrame
print(df.tail()) #print(df.head())

# Export the modified Feature Collection to Google Earth Engine Asset
export_task = ee.batch.Export.table.toAsset(
    collection=merged2023_class_with_properties,
    description='Export Crop_normalized_2023 with classes',
    assetId='projects/ee-kkidia3/assets/groundnut_class_check9' #i will change based on Crop_2023_norm_bands_class
)

export_task.start()

print("Export task started. Check the Earth Engine Code Editor for task status.")


In [ ]:
# Export the modified Feature Collection to Google Drive as a shapefile
export_task = ee.batch.Export.table.toDrive(
    collection=merged2023_class_with_properties,
    description='Export Crop_normalized_2023 with classes',
    fileFormat='SHP',
    folder='crop',
    fileNamePrefix='groundnut_class_check5'  # Change based on Crop_2023_norm_bands_class
)

export_task.start()

### Edit column of the polygon and rows

In [ ]:
# gnut_class = gnut_2023_with_bands
# # Function to add the new properties to each feature
# def add_properties(feature):
#     return feature.set('class', 'crop').set('year', '2023').set('region', '???')#.set('groundnut', 'groundnut') # change any column

# # Map the function over the Feature Collection
# gnut_class_with_properties = gnut_class.map(add_properties)

# # Get the first few features to display their properties
# features_to_display =gnut_class_with_properties.limit(5).getInfo()['features']

# # Extract properties into a list of dictionaries
# properties_list = [feature['properties'] for feature in features_to_display]
# # Print the properties of the first few features
# #for feature in features_to_display:
#    # print(feature['properties'])

# # Create a DataFrame from the properties list
# df = pd.DataFrame(properties_list)



# # Remove the column
# #df = df.drop(columns=['year'], errors='ignore') #remove a column e.g ['timestamp']


# # Adding a new row
# #new_row = {'class': 'crop', 'groundnut': 'new_groundnut', 'other_column': 'value'}
# #df = df.append(new_row, ignore_index=True)

# # Remove a row by index (e.g., removing the first row)
# #df = df.drop(index=[0]) #0 = the first row

# # Display the DataFrame
# #print(df)

# df.head() #show it in data fram



### Export the modified polygon to GEE ASSET

Test the exported GEE asset

In [ ]:
#test
xx = load_crop_data('projects/ee-kkidia3/assets/groundnut_class_check5')
# Get the first few features to display their properties
f_display =xx.limit(5).getInfo()['features']

# Crop_2023_norm_bands_class = load_crop_data('projects/ee-kkidia3/assets/Crop_2023_No_norm_bands_class')
# # Get the first few features to display their properties
# f_display =Crop_2023_norm_bands_class.limit(5).getInfo()['features']

# Extract properties into a list of dictionaries
p_list = [feature['properties'] for feature in f_display]
# Print the properties of the first few features
#for feature in features_to_display:
   # print(feature['properties'])

# Create a DataFrame from the properties list
df = pd.DataFrame(p_list)

df.head() #show it in data fram

Sample randomforest calssification only gnut

### Visulaize the polygons in map

In [ ]:
# @title Default title text
# Define a function to add Earth Engine vector data as a layer to a Folium map
def add_ee_vector_layer(feature_collection, style, layer_name, map_object):
    painted = ee.Image().paint(feature_collection, 'constant', 2)  # Here 'constant' is a dummy property for visualization
    map_id_dict = painted.getMapId(style)
    folium.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; Google Earth Engine',
        name=layer_name,
        overlay=True,
        control=True
    ).add_to(map_object)

# Create a Folium map object
center = [14.4974, -14.4524]  # Center of the map in  Senegal
m = folium.Map(location=center, zoom_start=7)

# Styling for the vector layer
style = {
    'color': 'blue',  # Line color
    'fillColor': '00000000',  # Fill color with opacity (00)
}

# Add the merged crop fields to the map
add_ee_vector_layer(merged2023, style, 'Merged Crops 2023', m)

# Add a layer control panel to the map
folium.LayerControl().add_to(m)

# Display the map
#m

Remove overlapping polygons from your merged feature collection. Overlaps is due to data collection

For example let me sort out only two polygons out of maney gnut polygons for closer look and simple analysis.

In [ ]:
# @title Default title text
# Select two specific polygons by their indices or properties. e.g gnut
# Here, I want to select the first two polygons.
poly_2 = gnut.toList(2)

#print(poly_2)

# Get the individual polygons
polygon1 = ee.Feature(poly_2.get(0))
polygon2 = ee.Feature(poly_2.get(1))

# Print the geometries of the selected polygons to verify.
print('Polygon 1_Blue:', polygon1.geometry().getInfo())
print('Polygon 2_Red:', polygon2.geometry().getInfo())


# Create a map centered at an arbitrary point @polygon1
map_center = [13.237198228125724,-14.715474917616827]  #set this to a blue selected @polygon1
m = folium.Map(location=map_center, zoom_start=30) #

# Define a function to add a feature to the folium map.
def add_feature_to_map(feature, map_obj, color):
    geom = feature.geometry().getInfo()
    coords = geom['coordinates']
    if geom['type'] == 'Polygon':
        folium.Polygon(locations=[(pt[1], pt[0]) for pt in coords[0]], color=color).add_to(map_obj)
    elif geom['type'] == 'MultiPolygon':
        for poly in coords:
            folium.Polygon(locations=[(pt[1], pt[0]) for pt in poly[0]], color=color).add_to(map_obj)

# Add polygons to the map
add_feature_to_map(polygon1, m, 'blue')
add_feature_to_map(polygon2, m, 'red')

# Display the map
m

In [ ]:
# Calculate the total area of polygons in square meters for Rainfed season 2023. Example groundnut
area = gnut.geometry().area().format('%.1f').getInfo()
print(f"Area M2: {area} square meters OR,")

# Calculate the area in hectares (1 hectare = 10,000 square meters)
area_ha = gnut.geometry().area().divide(10000).format('%.1f').getInfo() # Convert square meters to hectares
print(f"Area Ha: {area_ha} hectares")

# Calculate the perimeter in meters
perimeter = gnut.geometry().perimeter().format('%.1f').getInfo()
print(f"Perimeter: {perimeter} meters")

# Get the centroid of the polygon
centroid = gnut.geometry().centroid().getInfo()
print(f"Centroid: {centroid}")

# Get the bounding box as a GeoJSON
bounding_box = gnut.geometry().bounds().getInfo()
print(f"Bounding Box: {bounding_box}")



### Estimate the each polygon area (m2), for each crop e.g Groundnt


In [ ]:
# Define a function to calculate the area of each polygon in hectares
#def calc_area(feature):
    #return feature.set('area_ha', ee.Number(feature.geometry().area().divide(10000)).format('%.3f'))

# Define a function to calculate the area of each polygon in m2
def calc_area(feature):
  return feature.set('area_m2', ee.Number(feature.geometry().area())
  .format('%.1f'))#.divide(10000)) & # .format('%.1f') use to limit the digits

# Apply the function to each feature in the collection
area_mapped = merged2023.map(calc_area)

# Fetch the mapped area information from the server
areas_info = area_mapped.limit(10).getInfo()  # Limiting to the first 10 features directly to show everthing remove .limit(10)

# Iterate through the first 10 features and print area in hectares with 3 decimal places
for feature in areas_info['features']:
    area = feature['properties']['area_m2'] ##m2
    print(f"Gnut Polygon_Area: {area} m2") #m2


### Area (ha) distributions e.g Groundnut

In [ ]:
# Define a function to calculate the area of each polygon in hectares
#def calc_area(feature):
    #return feature.set('area_ha', feature.geometry().area().divide(10000))  # Convert from square meters to hectares

# Define a function to calculate the area of each polygon in hectares
def calc_area(feature):
    return feature.set('area_ha', feature.geometry().area().divide(10000)) #--> # Convert from square meters to hectares

# Apply the function to each feature in the collection
area_mapped = gnut.map(calc_area)

# Extract and print the area of each polygon
areas_info = area_mapped.getInfo()

# Extracting areas into a list for plotting
areas = [feature['properties']['area_ha'] for feature in areas_info['features']]

# Plotting the distribution of polygon areas
plt.figure(figsize=(10, 6))
plt.hist(areas, bins='auto', color='skyblue', alpha=0.8, rwidth=0.85)
plt.title('Distribution of Polygon Areas in Ha for Groundnut')
plt.xlabel('Area (Ha)')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

Show area of each crop

In [ ]:
#  'gnut', 'gsorrel', 'rice', etc. are already defined as ee.Geometry() or ee.Feature() objects representing each crop
crop_types = {
    'Gnut': gnut,
    'G. Sorrel': gsorrel,
    'Rice': rice,
    'Sesame': sesame,
    'Soye': soye,
    'Watermelon': melon,
    'Cassava': cassava,
    'Gnut Mix': gnutmix,
    'Millet Mix': milletmix,
    'Okra': okra,
    'Sorgh': sorgh,
    'Wheat': wheat,
    'Millet': millet,
    'Cowpea Mix': cowpmix,
    'Fallow': fallow
}

# Calculate area for each crop and store in a dictionary
#areas = {name: crop.geometry().area().divide(10000).getInfo() for name, crop in crop_types.items()} #In Hectares
areas = {name: crop.geometry().area().getInfo() for name, crop in crop_types.items()}


# Create a DataFrame from the area dictionary
area_df = pd.DataFrame(list(areas.items()), columns=['Crop Type', 'Area (Squere meteres M2)'])

# Display the DataFrame
print(area_df)

area_df.head(14)

Graphic of each crop in area coverages

In [ ]:
area_df = pd.DataFrame(list(areas.items()), columns=['Crop Type', 'Area (M2)'])

# Sort the DataFrame by area for better visualization
area_df.sort_values('Area (M2)', ascending=False, inplace=True)

# Plotting
plt.figure(figsize=(12, 8))
plt.bar(area_df['Crop Type'], area_df['Area (M2)'], color='red')
plt.xlabel('Crop Type')
plt.ylabel('Area in M2')
plt.title('Area of Each Crop Type for FY2023')
plt.xticks(rotation=90)  # Rotate crop type names for better readability
plt.grid(False)
plt.show()

### Count Pixels (NICFI) for each crop field represnetation of each specteral bands e.g Groundnut

In [ ]:
# time range
start_date = '2023-09-01'
end_date = '2023-09-30'

# Load the Planet/NICFI imagery
imagery = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/africa').filterDate(start_date, end_date).filterBounds(merged2023)  # Assuming 'gnut' is the ee.Geometry() of my area of interest
#bands info
bands = ee.ImageCollection(imagery).first()
print(bands.bandNames().getInfo())

#################  Planet Pixel count ####### Actual pixel count from PLANET

# Function to mask and count pixels within the 'gnut' polygon
def count_pixels(image):
    # Mask the image with the polygon
    masked_image = image.clip(gnut) #remember 1 polygone selected for closer see demo

    # Count pixels - 4.77 X 4.77 m (~5X5m) actual Planet/NICFI pixel resolution
    pixel_count = masked_image.reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=gnut,
        scale=4.77,  # Scale in meters; 5m resolution take time to estimate (timeout)
        # to avoid time out estimation use 10X10m and convert it in to 5X5 by devide the result 4
        maxPixels=1e9  # Adjust maxPixels if needed to handle large areas
    )

    # Need to return an ee.Feature for the .map() function
    return ee.Feature(None, pixel_count)

# Apply the pixel counting to each image in the collection
pixel_counts = imagery.map(count_pixels)

# Get information from the ImageCollection
pixel_counts_info = pixel_counts.getInfo()

# Print out the results
for feature in pixel_counts_info['features']:
    print(feature['properties'])



In [ ]:
# Define the time range
start_date = '2023-09-01'
end_date = '2023-09-30'


# Load the Planet/NICFI imagery
imagery = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/africa') \
            .filterDate(start_date, end_date) \
            .filterBounds(merged2023)

# Print band information
bands = imagery.first()
print(bands.bandNames().getInfo())

# Function to mask and count pixels within the AOI
def count_pixels(image):
    # Mask the image with the polygon
    masked_image = image.clip(merged2023)

    # Count pixels - 10m resolution for faster computation
    pixel_count = masked_image.reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=merged2023,
        scale=10,  # Scale in meters; coarser resolution for faster computation
        maxPixels=1e6  # Adjust maxPixels if needed to handle large areas
    )

    # Need to return an ee.Feature for the .map() function
    return ee.Feature(None, pixel_count)

# Apply the pixel counting to each image in the collection
pixel_counts = imagery.map(count_pixels)

# Get information from the ImageCollection
pixel_counts_info = pixel_counts.getInfo()

# Print out the results
for feature in pixel_counts_info['features']:
    print(feature['properties'])

### pixel esimates for each crop considering area not pixels

In [ ]:
# Assuming these are the individual crop polygons. others to be added from other crop years.
crop_types = {
    'gnut': gnut, 'gsorrel': gsorrel, 'rice': rice, 'sesame': sesame,
    'soye': soye, 'watermelon': melon, 'cassava': cassava,
    'gnutmix': gnutmix, 'milletmix': milletmix, 'okra': okra,
    'sorgh': sorgh, 'wheat': wheat, 'millet': millet, 'cowpmix': cowpmix,
    'fallow': fallow
}

def count_pixels(crop_name, crop_polygon):
    # Filter the imagery to the bounds of the crop polygon
    crop_imagery = imagery.filterBounds(crop_polygon)

    # Apply the pixel counting for each image in the filtered collection
    def pixel_count(image):
        masked_image = image.clip(crop_polygon)
        pixel_count = masked_image.reduceRegion(
            reducer=ee.Reducer.count(),
            geometry=crop_polygon,
            scale=10,  # Adjust the scale based on the actual resolution of NICFI imagery
            maxPixels=1e9
        )
        return ee.Feature(None, pixel_count)

    pixel_counts = crop_imagery.map(pixel_count)
    pixel_counts_info = pixel_counts.getInfo()

    # Print results for each crop type
    print(crop_name)
    for feature in pixel_counts_info['features']:
        print(feature['properties'])

# Run the pixel counting for each crop type
for crop_name, crop_polygon in crop_types.items():
    count_pixels(crop_name, crop_polygon)

### Pixel Exclusion representes less (e.g 40%) with in inside the polygon area.

Masking Pixels: Pixels with less than 40% coverage by the polygon are excluded using the mask method, where mask = fraction.gte(0.4) creates a mask that includes only pixels where the fraction is 40% or more.

- Band Selection: When creating the mask, the code now ensures that a single band is selected using image.select(0). This selects the first band of the image, assuming the first band is suitable for your masking purposes. Adjust the band selection if necessary based on the bands available in your specific imagery.

- Mask Application: By using single_band_image.mask() in the fraction calculation, we ensure that any operations involving masks deal with a single-band image, thus avoiding band mismatch errors.

In [ ]:
# Function to mask and count pixels within the 'gnut' polygon
def count_pixels(image):
    # Ensure that we work with a single band for mask creation
    # Here, you could potentially select a specific band or reduce to a single band
    # As an example, if the image has multiple bands, use just one (e.g., the first band) for mask calculations
    single_band_image = image.select(1)

    # Calculate the fraction of the pixel area that overlaps with the polygon using a constant image
    constantImage = ee.Image.constant(1).clip(okra)
    fraction = constantImage.updateMask(single_band_image.mask()).reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=gnut,
        scale=5,  # estimation with planet 5x5 is very difficult
        maxPixels=1e9
    ).get('constant')

    # Threshold the fraction to include only pixels with >= 40% coverage
    mask = ee.Image(fraction).gte(0.4)

    # Mask the image with the calculated mask
    masked_image = image.updateMask(mask)

    # Count pixels
    pixel_count = masked_image.reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=gnut,
        scale=5,
        maxPixels=1e9
    )

    # Return an ee.Feature for the .map() function to work properly
    return ee.Feature(None, pixel_count)

# Apply the pixel counting to each image in the collection
pixel_counts = imagery.map(count_pixels)

# Get information from the ImageCollection
pixel_counts_info = pixel_counts.getInfo()

# Print out the results
for feature in pixel_counts_info['features']:
    print(feature['properties'])

only one polygon from gnut only for learning use

In [ ]:
# Function to mask and count pixels within the 'gnut' polygon
def count_pixels(image):
    # Ensure that we work with a single band for mask creation
    single_band_image = image.select(1)  # Select the first band (assuming 1-based index)

    # Calculate the fraction of the pixel area that overlaps with the polygon using a constant image
    constant_image = ee.Image.constant(1).clip(polygon1)
    fraction = constant_image.updateMask(single_band_image.mask()).reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=gnut.geometry(),
        scale=5,  # estimation with planet 5x5 is very difficult
        maxPixels=1e9
    ).get('constant')

    # Threshold the fraction to include only pixels with >= 40% coverage
    mask = ee.Image.constant(fraction).gte(0.4)

    # Mask the image with the calculated mask
    masked_image = image.updateMask(mask)

    # Count pixels
    pixel_count = masked_image.reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=gnut.geometry(),
        scale=5,
        maxPixels=1e9
    )

    # Return an ee.Feature for the .map() function to work properly
    return ee.Feature(None, pixel_count)

# Apply the pixel counting to each image in the collection
pixel_counts = imagery.map(count_pixels)

# Get information from the ImageCollection
pixel_counts_info = pixel_counts.getInfo()

# Print out the results
for feature in pixel_counts_info['features']:
    print(feature['properties'])

In [ ]:
# Function to mask and count pixels within the 'gnut' polygon
def count_pixels(image):
    # Create a single band composite by averaging RGB and NIR bands
    composite = image.expression(
        "(R + G + B + N) / 4",  # Expression combining the bands
        {
            'R': image.select('R'),  #  'R' is the Red band
            'G': image.select('G'),  # 'G' is the Green band
            'B': image.select('B'),  #  'B' is the Blue band
            'N': image.select('N')   #  'N' is the NIR band
        }
    )

    # Generate a mask based on the composite value, adjusting threshold as necessary
    mask = composite.gt(0.2)  # Example threshold

    # Mask the image with the generated mask
    masked_image = image.updateMask(mask)

    # Count pixels
    pixel_count = masked_image.reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=gnut,
        scale=5,
        maxPixels=1e9
    )

    # Return an ee.Feature for the .map() function to work properly
    return ee.Feature(None, pixel_count)

# Apply the pixel counting to each image in the collection
pixel_counts = imagery.map(count_pixels)

# Get information from the ImageCollection
pixel_counts_info = pixel_counts.getInfo()

# Print out the results
for feature in pixel_counts_info['features']:
    print(feature['properties'])

Pixel counts using area coverage regardless of the specteral bands (NICFI + 4.77m X 4.77 M resolutions)

In [ ]:
resolution = 4.77  # Resolution of NICFI imagery in meters (4.77m x 4.77m per pixel)


# Function to calculate pixel count
def calculate_pixels(crop):
    area = crop.geometry().area()  # Area in square meters
    pixel_area = resolution * resolution  # Area of one pixel in square meters
    pixel_count = area.divide(pixel_area)  # Number of pixels
    return pixel_count.getInfo()

# Calculate pixel count for each crop and store in a dictionary
pixel_counts = {name: calculate_pixels(crop) for name, crop in crop_types.items()}

# Create a DataFrame from the area and pixel count dictionaries
area_df = pd.DataFrame(list(pixel_counts.items()), columns=['Crop Type', 'Pixel Count'])

# Sort the DataFrame by 'Pixel Count' in descending order
area_df = area_df.sort_values(by='Pixel Count', ascending=False)
# Display the DataFrame
print(area_df)

In [ ]:
# Plotting
#plt.figure(figsize=(10, 6))
#plt.bar(area_df['Crop Type'], area_df['Pixel Count'], color='green')
#plt.xlabel('Crop Type')
#plt.ylabel('Pixel Count')
#plt.title('Pixel Count by Crop Type (Ascending Order)')
#plt.xticks(rotation=45, ha='right')
#plt.tight_layout()  # Adjust layout to make room for the rotated x-axis labels
#plt.show()
####################
# Plotting using seaborn lib.
plt.figure(figsize=(12, 8))
sns.barplot(x='Crop Type', y='Pixel Count', data=area_df, palette='viridis')  # Using 'viridis' palette for varying colors
plt.xlabel('Crop Type')
plt.ylabel('Pixel Count')
plt.title('Pixel Count by Crop Type - for data base 2(FY2023)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()  # Adjust layout to make room for the rotated x-axis labels
plt.show()

Area vs Pixel count

In [ ]:
# Assuming 'crop_types' dictionary is predefined with ee.Geometry() objects
resolution = 4.77  # Resolution of NICFI imagery in meters

# Function to calculate pixel count
def calculate_pixels(area):
    pixel_area = resolution * resolution  # Area of one pixel in square meters
    return area / pixel_area

# Calculate area and pixel count for each crop and store in dictionaries
area_and_pixels = []
for name, crop in crop_types.items():
    area = crop.geometry().area().getInfo()  # Area in square meters
    pixel_count = calculate_pixels(area)
    area_and_pixels.append((name, area, pixel_count))

# Create a DataFrame from the results
df = pd.DataFrame(area_and_pixels, columns=['Crop Type', 'Area (Square Meters)', 'Pixel Count'])

# Sort the DataFrame by 'Pixel Count' in descending order (optional)
df.sort_values(by='Pixel Count', ascending=False, inplace=True)

# Plotting
fig, ax1 = plt.subplots(figsize=(14, 8))

# Bar plot for Area using Seaborn
color = 'tab:red'
sns.barplot(x='Crop Type', y='Area (Square Meters)', data=df, palette='viridis', ax=ax1)
ax1.set_xlabel('Crop Type')
ax1.set_ylabel('Area (Square Meters)', color=color)
ax1.tick_params(axis='y', labelcolor=color)

# Create a twin Axes sharing the x-axis for Pixel Count
ax2 = ax1.twinx()
color = 'tab:blue'
sns.lineplot(x='Crop Type', y='Pixel Count', data=df, sort=False, marker='o', color='red', ax=ax2)
ax2.set_ylabel('Pixel Count', color=color)
ax2.tick_params(axis='y', labelcolor=color)

# Improve layout and set x-axis labels rotation
plt.xticks(rotation=90)
fig.tight_layout()

plt.show()

### Garba

Hi Garba, please use the above functions and generate the mean/median pixel specteral band values for "R", "G", "B" and "N" values of each pixel for each crop. Let me know if you need my assitance or have questions on the above functions. Perhaps you can show it in vector/tabular form.

--------------------------------
R      |    G  |     B  |     N
-------------------------------
       |       |        |  

**Steps**

To enhance the quality and usability of the images for further analysis or applications Normalization and calibration of images are essential processes in image processing and computer vision.  

**1. Data Collection**: Obtain multispectral images of the crops that include the required spectral bands (R, G, B, and NIR).

**2. Image Preprocessing**:

2.1. Image Normalization

In [ ]:


# Load the image
image = cv2.imread('/path/to/your/image.jpg')

# Convert BGR image to RGB
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Normalize the image to range [0, 1]
normalized_image = image / 255.0

# Alternatively, you can normalize to range [-1, 1]
# normalized_image = (image / 127.5) - 1.0

# Display the original and normalized images
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(image)

plt.subplot(1, 2, 2)
plt.title('Normalized Image')
plt.imshow(normalized_image)

plt.show()


2.2. Calibration

In [ ]:
import cv2
import numpy as np
import glob

# Termination criteria for corner sub-pixel accuracy
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Prepare object points (0,0,0), (1,0,0), (2,0,0), ..., (6,5,0)
objp = np.zeros((6*7, 3), np.float32)
objp[:, :2] = np.mgrid[0:7, 0:6].T.reshape(-1, 2)

# Arrays to store object points and image points from all images
objpoints = []  # 3d points in real world space
imgpoints = []  # 2d points in image plane

# Load calibration images
images = glob.glob('/path/to/calibration/images/*.jpg')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (7, 6), None)

    # If found, add object points, image points (after refining them)
    if ret:
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (7, 6), corners2, ret)
        cv2.imshow('img', img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

# Perform camera calibration
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# Save the calibration results
np.savez('calibration_data.npz', mtx=mtx, dist=dist, rvecs=rvecs, tvecs=tvecs)

# Undistort an image using the calibration results
img = cv2.imread('/path/to/your/test/image.jpg')
h, w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))

# Undistort
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

# Crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]

# Display the result
cv2.imshow('calibrated', dst)
cv2.waitKey(0)
cv2.destroyAllWindows()


2.3. Alignment

  **  Install OpenCV**: Install OpenCV in your Colab environment to use its functionalities.

    **Load and Display Images**: Load the two images you want to align. Display them using Matplotlib to ensure they are loaded correctly.

    **Detect ORB Features and Compute Descriptors**: Use the ORB detector to find keypoints and compute descriptors for both images.

    **Match Features Using BFMatcher**: Match the descriptors using BFMatcher, and sort the matches based on their distance (quality).

    **Find Homography and Warp Image**: Extract the coordinates of the matched points, compute the homography matrix using RANSAC, and apply the homography to warp the second image to align it with the first image.

1. **Install OpenCV**: First, ensure you have OpenCV installed in your Colab environment.

In [ ]:
!pip install opencv-python-headless


**2. Load and Display Images**: Load the images you want to align.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the images in grayscale
img1 = cv2.imread('/path/to/your/image1.jpg', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('/path/to/your/image2.jpg', cv2.IMREAD_GRAYSCALE)

# Display the images
plt.subplot(1, 2, 1)
plt.title('Image 1')
plt.imshow(img1, cmap='gray')

plt.subplot(1, 2, 2)
plt.title('Image 2')
plt.imshow(img2, cmap='gray')

plt.show()


**3. Detect ORB Features and Compute Descriptor**s:

In [ ]:
# Initialize the ORB detector
orb = cv2.ORB_create()

# Detect keypoints and compute descriptors
keypoints1, descriptors1 = orb.detectAndCompute(img1, None)
keypoints2, descriptors2 = orb.detectAndCompute(img2, None)


**4. Match Features Using BFMatcher**:

In [ ]:
# Create BFMatcher object
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Match descriptors
matches = bf.match(descriptors1, descriptors2)

# Sort them in the order of their distance
matches = sorted(matches, key=lambda x: x.distance)

# Draw top matches
img_matches = cv2.drawMatches(img1, keypoints1, img2, keypoints2, matches[:50], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

# Display the matches
plt.figure(figsize=(20, 10))
plt.imshow(img_matches)
plt.title('Feature Matches')
plt.show()


**5. Find Homography and Warp Image:**

In [ ]:
# Extract location of good matches
points1 = np.zeros((len(matches), 2), dtype=np.float32)
points2 = np.zeros((len(matches), 2), dtype=np.float32)

for i, match in enumerate(matches):
    points1[i, :] = keypoints1[match.queryIdx].pt
    points2[i, :] = keypoints2[match.trainIdx].pt

# Find homography matrix
h, mask = cv2.findHomography(points2, points1, cv2.RANSAC)

# Use homography to warp the image
height, width = img1.shape
aligned_img = cv2.warpPerspective(img2, h, (width, height))

# Display the aligned image
plt.figure(figsize=(10, 10))
plt.subplot(1, 2, 1)
plt.title('Reference Image')
plt.imshow(img1, cmap='gray')

plt.subplot(1, 2, 2)
plt.title('Aligned Image')
plt.imshow(aligned_img, cmap='gray')

plt.show()


**Extract Pixel Values: Extract the pixel values for each spectral band (R, G, B, NIR) for each segmented crop area.**

Clipping

3. **Segmentation**

**4. Extract Pixel Values**: Extract the pixel values for each spectral band (R, G, B, NIR) for each segmented crop area.

**5. Compute Statistics:**

    Mean: Calculate the mean value for each spectral band across all pixels for each crop.
    Median: Calculate the median value for each spectral band across all pixels for each crop.

### Cyrille


**GARBA**  Other Proposition

[**Click Here for the first step to follow**](https://code.earthengine.google.com/91904df41cc77089f346977c173a0122)

Change the variable to our variable

1.   Area of Interest(roi)
2.   Land cover entities
3.   Filter the date
4.   Setup the other parameters for the convenance  


Install some packages


In [ ]:
!pip install rasterio
!pip install earthpy



In [ ]:
#Import packages
import pandas as pd
import numpy as np
import keras
from keras import sequential
from keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, Flatten, Input, GlobalMawPooling1D
from keras.callbacks import EarlyStopping
from kera import Model
import rasterio
import earth.plot as ep
from keras.utils import to_categorical
import sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, Classification_report
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import from_levels_and_colors

In [ ]:
#Mount GDRIVE
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at/content/drive; to attemp to forcibly remount, call drive.mount("/content/drive" force_remount=True)

In [ ]:
# Parameter
FEATURES = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'EVI', 'NBR', 'NDMI', 'NDWI', 'NDBI', 'NDBaI', 'elevation']
LABEL = ['classvalue']
SPLIT = ['sample']
N_CLASSES = 9
CLASSES = [1, 2, 3, 4, 5, 6, 7, 8, 9]
PALETTE = ['#F08080', '#D2B48C', '#87CEFA', '#008080', '#90EE90', '#228B22', '#808000', '#FF8C00', '#006400']
SAMPLE_PATH = '/content/drive/MyDrive/DL/Samples_LC_Jambi_2023.csv'
IMAGE_PATH = '/content/drive/MyDrive/DL/Landsat_Jambi_2023.tif'

In [ ]:
# Load image
image = rasterio.open(IMAGE_PATH)
bandNum = image.count
height = image.height
width = image.width
crs = image.crs
transform = image.transform
shape = (height, width)

image_vis = []
for x in [6, 5, 4]:
  image_vis.append(image.read(x))
image_vis = np.stack(image_vis)

plot_size = (8, 8)
ep.plot_rgb(
  image_vis,
  figsize=plot_size,
  stretch=True,
)

In [ ]:
# Read sample
samples = pd.read_csv(SAMPLE_PATH)
samples = samples.sample(frac = 1) # Shuffle data
samples

In [ ]:
# Split into train and test based on column
train = samples[samples['sample'] == 'train']
test = samples[samples['sample'] == 'test']

# Split between features and label
train_features = train[FEATURES]
train_label = train[LABEL]
test_features = test[FEATURES]
test_label = test[LABEL]

# Function to reshape array input
def reshape_input(array):
  shape = array.shape
  return array.reshape(shape[0], shape[1], 1)

# Convert samples dataframe (pandas) to numpy array
train_input = reshape_input(train_features.to_numpy())
test_input = reshape_input(test_features.to_numpy())

# Also make label data to categorical
train_output = to_categorical(train_label.to_numpy(), N_CLASSES + 1, int)
test_output = to_categorical(test_label.to_numpy(), N_CLASSES + 1, int)

# Show the data shape
print(f'Train features: {train_input.shape}\nTest features: {test_input.shape}\nTrain label: {train_output.shape}\nTest label: {test_output.shape}')

Train features: (14853, 14, 1)
Test features: (4043, 14, 1)
Train label: (14853, 10)
Test label: (4043, 10)

In [ ]:
# Make model for our data
# Input shape
train_shape = train_input.shape
input_shape = (train_shape[1], train_shape[2])

# Model parameter
neuron = 64
drop = 0.2
kernel = 2
pool = 2

# Make sequential model
model = Sequential([
  Input(input_shape),
  Conv1D(neuron * 1, kernel, activation='relu'),
  Conv1D(neuron * 1, kernel, activation='relu'),
  MaxPooling1D(pool),
  Dropout(drop),
  Conv1D(neuron * 2, kernel, activation='relu'),
  Conv1D(neuron * 2, kernel, activation='relu'),
  MaxPooling1D(pool),
  Dropout(drop),
  GlobalMaxPooling1D(),
  Dense(neuron * 2, activation='relu'),
  Dropout(drop),
  Dense(neuron * 1, activation='relu'),
  Dropout(drop),
  Dense(N_CLASSES + 1, activation='softmax')
])

model.summary()

In [ ]:
# Train the model

# Compline the model
model.compile(
    optimizer='Adam',
    loss='CategoricalCrossentropy',
    metrics=['accuracy']
)

# Create callback to stop training if loss not decreasing
stop = EarlyStopping(
    monitor='loss',
    patience=5
)

# Fit the model
result = model.fit(
    x=train_input, y=train_output,
    validation_data=(test_input, test_output),
    batch_size=1024,
    callbacks=[stop],
    epochs=100,
)

In [ ]:
# Show history
history = pd.DataFrame(result.history)

plt.figure(figsize = (10, 8))
plt.plot(range(len(history['accuracy'].values.tolist())), history['accuracy'].values.tolist(), label = 'Train_Accuracy')
plt.plot(range(len(history['loss'].values.tolist())), history['loss'].values.tolist(), label = 'Train_Loss')
plt.plot(range(len(history['val_accuracy'].values.tolist())), history['val_accuracy'].values.tolist(), label = 'Test_Accuracy')
plt.plot(range(len(history['val_loss'].values.tolist())), history['val_loss'].values.tolist(), label = 'Test_Loss')
plt.xlabel('Epochs')
plt.ylabel('Value')
plt.legend()
plt.show()

In [ ]:
# Predict test data
prediction = np.argmax(model.predict(test_input), 1).flatten()
label = np.argmax(test_output, 1).flatten()

# Confusion matrix
cm = confusion_matrix(label, prediction, normalize='true')
cm = ConfusionMatrixDisplay(cm)
cm.plot()

# Classification report
print(classification_report(label, prediction))

In [ ]:
# Predict image using the model
image_input = []
for x in range(14):
  image_input.append(image.read(x + 1))
image_input = reshape_input(np.stack(image_input).reshape(14, -1).T)

# Predict
prediction = model.predict(image_input, batch_size=4096*20)
prediction = np.argmax(prediction, 1)
prediction = prediction.reshape(shape[0], shape[1])

# Visualize
cmap, norm = from_levels_and_colors(CLASSES, PALETTE, extend='max')
ep.plot_bands(prediction, cmap=cmap, norm=norm, figsize=plot_size)

In [ ]:
# Save file to drive
save_location = '/content/drive/MyDrive/DL/'
name = 'LC_Jambi_2023.tif'
location = save_location + name

new_dataset = rasterio.open(
      location,
      mode='w', driver='GTiff',
      height = prediction.shape[0], width = prediction.shape[1],
      count=1, dtype=str(prediction.dtype),
      crs=crs,
      transform=transform
)
new_dataset.write(prediction, 1);
new_dataset.close()

For following step by step [Youtube Chanel](https://www.youtube.com/watch?v=NFoZPyQqVRA) **source**: Ramadhan